Index and test of serach using BM25

In [1]:
!pip install python-terrier
import pyterrier as pt
import pandas as pd
pt.init()

Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
<ipython-input-1-df7663096205>:4: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


In [14]:
!rm -rf ./index

# Make sure this dataset exist before running
df = pd.read_csv("cleaned_dataset.csv")

# Make sure the dataset contains columns of 'docno' and 'text'
if 'docno' not in df.columns or 'text' not in df.columns:
    raise ValueError("CSV file must include columns of 'docno' 和 'text' ")

# Transfer dataframe to dictionary form
doc_iter = df.to_dict(orient="records")

# Create Indexer
indexer = pt.IterDictIndexer("./index", meta={'docno': 20, 'text': 4096}, fields=['text'])

# Genrate Index
indexref = indexer.index(doc_iter)

# load index and print it
index = pt.IndexFactory.of(indexref)
print(index.getCollectionStatistics().toString())

Number of documents: 3009
Number of terms: 11620
Number of postings: 164596
Number of fields: 1
Number of tokens: 253171
Field names: [text]
Positions:   false



A simple test run of BM25(Search US president)

In [15]:
result_df1 = pt.BatchRetrieve(indexref, wmodel="BM25").search("US president")
result_df1

<ipython-input-15-7289b9a150d9>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  result_df1 = pt.BatchRetrieve(indexref, wmodel="BM25").search("US president")


,qid,docid,docno,rank,score,query
0,1,514,"""5529""",0,6.721623,US president
1,1,1867,"""6735""",1,6.329596,US president
2,1,1372,"""5889""",2,6.228903,US president
3,1,2045,"""7222""",3,6.178902,US president
4,1,2056,"""7256""",4,6.178902,US president
...,...,...,...,...,...,...
198,1,1509,"""6209""",198,1.571240,US president
199,1,2605,"""21359""",199,1.555545,US president
200,1,1847,"""6601""",200,1.404067,US president
201,1,2976,"""19928""",201,1.352835,US president


In [16]:
result_df2 = pt.BatchRetrieve(indexref, wmodel="BM25F", properties = {'w.0': 0.5, 'w.1': 1}).search("US president")
result_df2

11:35:44.950 [main] WARN org.terrier.matching.models.PerFieldNormWeightingModel -- Weighting model BM25F is invoked on an index with only 1 field - this is probably incorrect.


<ipython-input-16-40c9a7ec8476>:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  result_df2 = pt.BatchRetrieve(indexref, wmodel="BM25F", properties = {'w.0': 0.5, 'w.1': 1}).search("US president")


,qid,docid,docno,rank,score,query
0,1,514,"""5529""",0,6.075545,US president
1,1,1867,"""6735""",1,5.775117,US president
2,1,2434,"""20972""",2,5.718562,US president
3,1,2045,"""7222""",3,5.451626,US president
4,1,2056,"""7256""",4,5.451626,US president
...,...,...,...,...,...,...
198,1,1509,"""6209""",198,0.713776,US president
199,1,2605,"""21359""",199,0.705156,US president
200,1,1847,"""6601""",200,0.623802,US president
201,1,2976,"""19928""",201,0.597016,US president
